In [1]:
import torch
import numpy as np
import torch.nn as nn
from torchvision import datasets, models, transforms as T
from PIL import Image


In [2]:
alexnet = models.alexnet(pretrained=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /Users/joachimsvendsen/.cache/torch/checkpoints/alexnet-owt-4df8aa71.pth
100%|██████████| 233M/233M [00:11<00:00, 21.3MB/s] 


In [3]:
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [5]:
#transform = T.Compose([T.Resize(256), T.CenterCrop(224), T.ToTensor()])
#dataset = datasets.ImageNet(".", split="train", transform=transform)
# sample execution (requires torchvision)
input_image = Image.open(filename)
preprocess = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# To GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    alexnet.to('cuda')

with torch.no_grad():
    output = alexnet(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
print(torch.nn.functional.softmax(output[0], dim=0))


tensor([-1.7581e+00, -4.4613e+00, -1.3566e+00, -4.2777e+00, -2.4732e+00,
         5.1095e-01,  3.6125e-01,  6.5426e+00,  9.8937e+00,  2.6503e+00,
        -3.2326e-01, -1.8935e-01,  9.8931e-01, -2.5838e-01, -1.9399e+00,
        -1.6543e+00,  1.1768e+00,  7.4438e-01,  1.9708e+00, -2.5417e+00,
        -4.2199e-01,  6.3383e+00, -5.5330e-01,  6.6968e+00,  8.7913e-02,
        -5.6637e+00, -5.1952e+00, -3.8658e+00, -4.7050e+00, -4.5498e-01,
        -3.3042e+00, -5.8526e+00, -5.0678e+00, -2.8549e+00, -9.8966e-01,
        -3.5618e+00,  4.9902e+00, -3.9088e+00, -5.0909e+00, -4.5216e+00,
        -4.3974e+00, -5.4490e+00, -6.3560e+00, -4.9477e+00, -3.2107e+00,
        -4.5208e+00,  2.3673e-01, -5.2982e+00, -5.8545e+00, -4.1687e+00,
        -4.0684e+00, -4.4629e+00, -5.0438e+00, -6.7722e+00, -3.4940e+00,
        -4.5713e+00, -2.9373e+00, -3.0750e+00, -5.9519e+00, -5.2649e+00,
        -3.1370e+00, -6.6981e+00, -6.7063e+00, -5.9919e+00, -5.2436e+00,
        -5.0557e+00, -4.7223e+00, -6.2605e+00, -4.2

# Fast Gradient Sign Method (FGSM)

$$
x^{a d v}=x+\varepsilon \cdot \operatorname{sign}\left(\nabla_{x} J\left(x, y_{t r u e}\right)\right)
$$

In [ ]:
# FGSM attack code
def fgsm_attack(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image